# HPXPy Distributed Analytics Demo

This notebook benchmarks HPXPy vs NumPy on a data science workflow, demonstrating HPXPy's parallel performance advantage on large datasets.

## Use Case: IoT Sensor Network Analytics

- Millions of sensor readings from a distributed sensor network
- Each reading has: timestamp, sensor_id, temperature, humidity, pressure
- Goal: Compute statistics, detect anomalies, compute derived features

In [ ]:
import time
import numpy as np
import hpxpy as hpx

hpx.init(num_threads=4)

## Simulate Sensor Data

In [ ]:
def simulate_sensor_data(n_samples):
    """Simulate IoT sensor network data."""
    np.random.seed(42)
    
    # Sensor readings with realistic patterns
    timestamps = np.arange(n_samples, dtype=np.float64)
    sensor_ids = np.random.randint(0, 1000, n_samples).astype(np.float64)
    
    # Temperature with daily cycle + noise
    base_temp = 20 + 10 * np.sin(2 * np.pi * timestamps / 86400)
    temperature = base_temp + np.random.normal(0, 2, n_samples)
    
    # Humidity inversely correlated with temperature
    humidity = 60 - 0.5 * (temperature - 20) + np.random.normal(0, 5, n_samples)
    humidity = np.clip(humidity, 0, 100)
    
    # Pressure with slow drift
    pressure = 1013 + 10 * np.sin(2 * np.pi * timestamps / 604800) + np.random.normal(0, 2, n_samples)
    
    return timestamps, sensor_ids, temperature, humidity, pressure

## NumPy Analytics Pipeline

In [ ]:
def benchmark_numpy(temperature, humidity, pressure, n_iterations=3):
    """Benchmark NumPy performance on analytics pipeline."""
    times = []
    
    for _ in range(n_iterations):
        start = time.perf_counter()
        
        # Statistics
        temp_mean = np.mean(temperature)
        temp_std = np.std(temperature)
        humid_mean = np.mean(humidity)
        humid_std = np.std(humidity)
        press_mean = np.mean(pressure)
        press_std = np.std(pressure)
        
        # Anomaly detection (z-score)
        temp_zscore = np.abs(temperature - temp_mean) / temp_std
        humid_zscore = np.abs(humidity - humid_mean) / humid_std
        press_zscore = np.abs(pressure - press_mean) / press_std
        
        n_temp_anomalies = np.sum(temp_zscore > 3.0)
        n_humid_anomalies = np.sum(humid_zscore > 3.0)
        n_press_anomalies = np.sum(press_zscore > 3.0)
        
        # Feature engineering
        heat_index = temperature * 1.8 + 32 + humidity * 0.1
        dew_point = temperature - ((100 - humidity) / 5)
        pressure_tendency = (pressure - press_mean) / press_std
        comfort = 100 - np.abs(temperature - 22) * 2 - np.abs(humidity - 50) * 0.5
        
        # Final reductions
        _ = np.mean(heat_index)
        _ = np.mean(dew_point)
        _ = np.mean(comfort)
        
        elapsed = time.perf_counter() - start
        times.append(elapsed)
    
    return min(times), {
        'temp_mean': temp_mean, 'temp_std': temp_std,
        'n_temp_anomalies': n_temp_anomalies,
        'n_humid_anomalies': n_humid_anomalies,
        'n_press_anomalies': n_press_anomalies
    }

## HPXPy Analytics Pipeline

In [ ]:
def benchmark_hpxpy(temp_hpx, humid_hpx, press_hpx, n_iterations=3):
    """Benchmark HPXPy performance on analytics pipeline."""
    times = []
    
    for _ in range(n_iterations):
        start = time.perf_counter()
        
        # Statistics
        temp_mean = hpx.mean(temp_hpx)
        temp_std = hpx.std(temp_hpx)
        humid_mean = hpx.mean(humid_hpx)
        humid_std = hpx.std(humid_hpx)
        press_mean = hpx.mean(press_hpx)
        press_std = hpx.std(press_hpx)
        
        # Anomaly detection (z-score)
        temp_zscore = hpx.abs(temp_hpx - temp_mean) / temp_std
        humid_zscore = hpx.abs(humid_hpx - humid_mean) / humid_std
        press_zscore = hpx.abs(press_hpx - press_mean) / press_std
        
        temp_anomaly = (temp_zscore > 3.0).to_numpy().astype(np.float64)
        humid_anomaly = (humid_zscore > 3.0).to_numpy().astype(np.float64)
        press_anomaly = (press_zscore > 3.0).to_numpy().astype(np.float64)
        
        n_temp_anomalies = int(hpx.sum(hpx.from_numpy(temp_anomaly)))
        n_humid_anomalies = int(hpx.sum(hpx.from_numpy(humid_anomaly)))
        n_press_anomalies = int(hpx.sum(hpx.from_numpy(press_anomaly)))
        
        # Feature engineering
        heat_index = temp_hpx * 1.8 + 32 + humid_hpx * 0.1
        dew_point = temp_hpx - ((100 - humid_hpx) / 5)
        pressure_tendency = (press_hpx - press_mean) / press_std
        comfort = 100 - hpx.abs(temp_hpx - 22) * 2 - hpx.abs(humid_hpx - 50) * 0.5
        
        # Final reductions
        _ = hpx.mean(heat_index)
        _ = hpx.mean(dew_point)
        _ = hpx.mean(comfort)
        
        elapsed = time.perf_counter() - start
        times.append(elapsed)
    
    return min(times), {
        'temp_mean': temp_mean, 'temp_std': temp_std,
        'n_temp_anomalies': n_temp_anomalies,
        'n_humid_anomalies': n_humid_anomalies,
        'n_press_anomalies': n_press_anomalies
    }

## Benchmark: Full Analytics Pipeline

In [ ]:
n_localities = hpx.distribution.get_num_localities()
locality_id = hpx.distribution.get_locality_id()

print(f"Configuration:")
print(f"  HPX Threads: 4")
print(f"  Localities: {n_localities}")
print(f"  Current locality: {locality_id}")

# Test multiple dataset sizes
sizes = [1_000_000, 5_000_000, 10_000_000]

print("\n" + "=" * 60)
print("Benchmark: Full Analytics Pipeline")
print("  (statistics + anomaly detection + feature engineering)")
print("=" * 60)

print(f"\n{'Dataset Size':>15} | {'NumPy (ms)':>12} | {'HPXPy (ms)':>12} | {'Speedup':>10}")
print("-" * 60)

for n_samples in sizes:
    # Generate data
    _, _, temperature, humidity, pressure = simulate_sensor_data(n_samples)
    
    # NumPy benchmark
    np_time, np_results = benchmark_numpy(temperature, humidity, pressure)
    
    # HPXPy benchmark
    temp_hpx = hpx.from_numpy(temperature)
    humid_hpx = hpx.from_numpy(humidity)
    press_hpx = hpx.from_numpy(pressure)
    
    hpx_time, hpx_results = benchmark_hpxpy(temp_hpx, humid_hpx, press_hpx)
    
    speedup = np_time / hpx_time
    
    print(f"{n_samples:>15,} | {np_time*1000:>12.2f} | {hpx_time*1000:>12.2f} | {speedup:>9.2f}x")
    
    # Verify correctness
    assert abs(np_results['temp_mean'] - hpx_results['temp_mean']) < 0.01
    assert np_results['n_humid_anomalies'] == hpx_results['n_humid_anomalies']

## Individual Operation Benchmarks

In [ ]:
n_samples = 10_000_000
_, _, temperature, humidity, pressure = simulate_sensor_data(n_samples)
temp_hpx = hpx.from_numpy(temperature)
humid_hpx = hpx.from_numpy(humidity)

operations = []

# Reduction: sum
start = time.perf_counter()
for _ in range(5):
    _ = np.sum(temperature)
np_sum = (time.perf_counter() - start) / 5

start = time.perf_counter()
for _ in range(5):
    _ = hpx.sum(temp_hpx)
hpx_sum = (time.perf_counter() - start) / 5
operations.append(('sum (reduction)', np_sum, hpx_sum))

# Element-wise: sqrt
start = time.perf_counter()
for _ in range(5):
    _ = np.sqrt(temperature + 50)
np_sqrt = (time.perf_counter() - start) / 5

temp_shifted = hpx.from_numpy(temperature + 50)
start = time.perf_counter()
for _ in range(5):
    _ = hpx.sqrt(temp_shifted)
hpx_sqrt = (time.perf_counter() - start) / 5
operations.append(('sqrt (element-wise)', np_sqrt, hpx_sqrt))

# Compute-heavy: chained math operations
start = time.perf_counter()
for _ in range(3):
    r = np.sin(temperature * 0.1)
    r = np.exp(r * 0.5)
    r = np.sqrt(np.abs(r))
    _ = np.sum(r)
np_heavy = (time.perf_counter() - start) / 3

start = time.perf_counter()
for _ in range(3):
    r = hpx.sin(temp_hpx * 0.1)
    r = hpx.exp(r * 0.5)
    r = hpx.sqrt(hpx.abs(r))
    _ = hpx.sum(r)
hpx_heavy = (time.perf_counter() - start) / 3
operations.append(('sin+exp+sqrt chain', np_heavy, hpx_heavy))

print(f"\n{'Operation':>25} | {'NumPy (ms)':>12} | {'HPXPy (ms)':>12} | {'Speedup':>10}")
print("-" * 70)

for op_name, np_t, hpx_t in operations:
    speedup = np_t / hpx_t
    print(f"{op_name:>25} | {np_t*1000:>12.2f} | {hpx_t*1000:>12.2f} | {speedup:>9.2f}x")

## Summary

### Key factors for HPXPy performance:

1. **SIMD vectorization** (`-march=native -ffast-math`)
2. **GIL released** during C++ execution
3. **Parallel element-wise operations** (`hpx::for_each`)
4. **Deterministic sequential reductions**

The real power comes from distributed execution across multiple nodes, where HPX's AGAS enables seamless data distribution.

In [ ]:
hpx.finalize()
print("Benchmark complete!")